# MNIST Large Untrained Net Exc Inh

Derived from https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb

In [118]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from keras import backend as K
#print("keras version:",tf.keras.__version__)

TensorFlow version: 2.8.2


In [119]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

In [120]:
inlineImplementation = False	#orig: True #True: excitatory/inhibitory neurons are on same sublayer, False: add inhibitory neurons to separate preceding sublayer
if(inlineImplementation):
    positiveWeightImplementation = False	#orig: True #optional
    if(not positiveWeightImplementation):
        integrateWeights = True    #orig: False #optional
        if(integrateWeights):
            integrateWeights1 = False    #explicitly declare E/I neurons
            integrateWeights2 = True    #implicitly declare E/I neurons 
            integrateWeightsInitialiseZero = False   #miscellaneous training performance improvement (single EI layer only)
else:
    positiveWeightImplementation = False    #False: only current coded implementation

inputLayerExcitatoryOnly = True #True: only current coded implementation

generateUntrainedNetwork = False
if(generateUntrainedNetwork):
    #only train the last layer
    numberOfHiddenLayers = 2    #default: 2    #if 0 then useSVM=True
else:
    numberOfHiddenLayers = 2 #default: 2

if(numberOfHiddenLayers > 1):
    addSkipLayers = False   #optional
else:
    addSkipLayers = False   #mandatory

layerSizeBase = 128  #default: 128

batch_size = 64 #default: 64
epochs = 100  #1  #5

debugNoEIneurons = False
debugPreTrainWeights = True
debugPreTrainOutputs = True
debugPostTrainWeights = True
debugPostTrainOutputs = True

## Load data

In [121]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print("x_train.shape = ", x_train.shape)

input_shape = (28, 28)

x_train.shape =  (60000, 28, 28)


## Define model

In [122]:
def EIactivation(x):
    return K.maximum(x, 0)  #ReLU
    
def EIactivationExcitatory(x):
    if(inlineImplementation):
        if(positiveWeightImplementation):
            return K.maximum(x, 0)  #ReLU
        else:
             print("EIactivationExcitatory error: requires positiveWeightImplementation")      
    else:
        print("EIactivationExcitatory error: requires inlineImplementation")

def EIactivationInhibitory(x):
    if(inlineImplementation):
        if(positiveWeightImplementation):
            return -(K.maximum(x, 0))   #ReLU with negative output
        else:
             print("EIactivationInhibitory error: requires positiveWeightImplementation")      
    else:
        print("inlineImplementation error: requires inlineImplementation")

def EIweightInitializer(shape, dtype=None):
    if(inlineImplementation):
        if(positiveWeightImplementation):
            w = tf.math.abs(tf.random.normal(shape, dtype=dtype))
        else:
            if(integrateWeights):
                if(integrateWeightsInitialiseZero):
                    w = tf.random.normal(shape, dtype=dtype)
                    #w = tf.zeros(shape, dtype=dtype)    #tf.math.abs(tf.random.normal(shape, dtype=dtype))
                else:
                    w = tf.math.abs(tf.random.normal(shape, dtype=dtype))
                    wEIsize = w.shape[0]//2
                    wSignE = tf.ones([wEIsize, w.shape[1]])
                    wSignI = tf.ones([wEIsize, w.shape[1]])
                    wSignI = tf.multiply(wSignI, -1)
                    wSign = tf.concat([wSignE, wSignI], axis=0)
                    w = tf.multiply(w, wSign)
            else:
                print("EIweightInitializer error: requires !positiveWeightImplementation:integrateWeights")
    else:
        print("EIweightInitializer error: requires inlineImplementation")

    return w

def EIweightInitializerExcitatory(shape, dtype=None):
    if(positiveWeightImplementation):
        print("EIweightInitializerExcitatory error: requires !positiveWeightImplementation")
    else:
        return tf.math.abs(tf.random.normal(shape, dtype=dtype))

def EIweightInitializerInhibitory(shape, dtype=None):
    if(positiveWeightImplementation):
        print("EIweightInitializerExcitatory error: requires !positiveWeightImplementation")
    else:
        return tf.math.negative(tf.math.abs(tf.random.normal(shape, dtype=dtype)))

class negative(tf.keras.constraints.Constraint):
    #based on https://www.tensorflow.org/api_docs/python/tf/keras/constraints/Constraint
    def __init__(self):
        pass
    def __call__(self, w):
        return w * tf.cast(tf.math.less_equal(w, 0.), w.dtype)

class positiveOrNegative(tf.keras.constraints.Constraint):
    #based on https://www.tensorflow.org/api_docs/python/tf/keras/constraints/Constraint
    def __init__(self):
        pass
    def __call__(self, w):
        w_shape = w.shape
        wEIsize = w.shape[0]//2
        wE = w[0:wEIsize]
        wI = w[wEIsize:]
        wEcheck = tf.greater_equal(wE, 0)
        wIcheck = tf.less_equal(wI, 0)
        wEcheck = tf.cast(wEcheck, tf.float32)
        wIcheck = tf.cast(wIcheck, tf.float32)
        wE = tf.multiply(wE, wEcheck)
        wI = tf.multiply(wI, wIcheck)
        w = tf.concat([wE, wI], axis=0)
        return w


In [123]:
num_classes = 10

if(inlineImplementation):
    if(positiveWeightImplementation):
        EIweightConstraint = tf.keras.constraints.non_neg()
        constrainBiases = True   #ensure positive biases also
        if(constrainBiases):
            EIbiasConstraint = tf.keras.constraints.non_neg()
            constrainBiasesLastLayer = False
            if(constrainBiasesLastLayer):
                EIbiasConstraintLastLayer = tf.keras.constraints.non_neg()
            else:
                EIbiasConstraintLastLayer = None
        else:
            EIbiasConstraint = None
            EIbiasConstraintLastLayer = None
        EIweightConstraintLastLayer = EIweightConstraint
    else:
        if(integrateWeights):
            EIweightConstraint = positiveOrNegative()
            EIbiasConstraint = None
            EIweightConstraintLastLayer = None
            EIbiasConstraintLastLayer = None
        else:
            EIweightConstraintPositive = tf.keras.constraints.non_neg()
            EIweightConstraintNegative = negative()
            constrainBiases = False
            if(constrainBiases):
                EIbiasConstraintPositive = tf.keras.constraints.non_neg()
                EIbiasConstraintNegative = negative()
            else:
                EIbiasConstraintPositive = None
                EIbiasConstraintNegative = None
            EIweightConstraintLastLayer = None
            EIbiasConstraintLastLayer = None
else:
    EIweightConstraintPositive = tf.keras.constraints.non_neg()
    EIweightConstraintNegative = negative()
    constrainBiases = False
    if(constrainBiases):
        EIbiasConstraintPositive = tf.keras.constraints.non_neg()
        EIbiasConstraintNegative = negative()
    else:
        EIbiasConstraintPositive = None
        EIbiasConstraintNegative = None
    EIweightConstraintLastLayer = None
    EIbiasConstraintLastLayer = None  

if(generateUntrainedNetwork):
    #only train the last layer
    generateLargeNetwork = True
else:
    generateLargeNetwork = False


if(generateLargeNetwork):
    generateLargeNetworkRatio = 50
    layerRatio = generateLargeNetworkRatio
else:
    layerRatio = 1  #10 #1

def createEIlayer(layerIndex, h0, firstLayer=False):
    if(debugNoEIneurons):
        h1E = tf.keras.layers.Dense(layerSizeBase*layerRatio)(h0)   #excitatory neuron inputs
        h1I = tf.keras.layers.Dense(layerSizeBase*layerRatio)(h0)   #inhibitory neuron inputs   
        h1E = tf.keras.layers.Activation(EIactivationExcitatory)(h1E)
        h1I = tf.keras.layers.Activation(EIactivationInhibitory)(h1I)
        h1 = tf.keras.layers.Concatenate()([h1E, h1I])  
    else:
        if(inlineImplementation):
            if(positiveWeightImplementation):
                h1E = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=EIweightInitializerExcitatory, kernel_constraint=EIweightConstraint, bias_constraint=EIbiasConstraint)(h0)
                h1I = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=EIweightInitializerInhibitory, kernel_constraint=EIweightConstraint, bias_constraint=EIbiasConstraint)(h0)
                h1E = tf.keras.layers.Activation(EIactivationExcitatory)(h1E)
                h1I = tf.keras.layers.Activation(EIactivationInhibitory)(h1I)
                h1 = tf.keras.layers.Concatenate()([h1E, h1I])
            else:
                if(integrateWeights):
                    if(integrateWeights1):
                        if(firstLayer):
                            h1E = tf.keras.layers.Dense(layerSizeBase*layerRatio)(h0)   #excitatory neuron inputs
                            h1I = tf.keras.layers.Dense(layerSizeBase*layerRatio)(h0)   #inhibitory neuron inputs             
                        else:
                            h1E = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=EIweightInitializer, kernel_constraint=EIweightConstraint, bias_constraint=EIbiasConstraint)(h0)   #excitatory neuron inputs
                            h1I = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=EIweightInitializer, kernel_constraint=EIweightConstraint, bias_constraint=EIbiasConstraint)(h0)   #inhibitory neuron inputs  
                        h1E = tf.keras.layers.Activation(EIactivation)(h1E)
                        h1I = tf.keras.layers.Activation(EIactivation)(h1I)
                        h1 = tf.keras.layers.Concatenate()([h1E, h1I])
                    elif(integrateWeights2):
                        if(firstLayer):
                            h1 = tf.keras.layers.Dense(layerSizeBase*layerRatio*4)(h0)   #excitatory neuron inputs
                        else:
                            h1 = tf.keras.layers.Dense(layerSizeBase*layerRatio*2, kernel_initializer=EIweightInitializer, kernel_constraint=EIweightConstraint, bias_constraint=EIbiasConstraint)(h0)   #excitatory neuron inputs
                        h1 = tf.keras.layers.Activation(EIactivation)(h1) #ReLU
                else:
                    if(firstLayer):
                        h1E = tf.keras.layers.Dense(layerSizeBase*layerRatio)(h0)   #excitatory neuron inputs
                        h1I = tf.keras.layers.Dense(layerSizeBase*layerRatio)(h0)   #inhibitory neuron inputs
                    else:
                        h0E, h0I = h0
                        h1Ee = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=EIweightInitializerExcitatory, kernel_constraint=EIweightConstraintPositive, bias_constraint=EIbiasConstraintPositive)(h0E) #excitatory neuron excitatory inputs
                        h1Ei = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=EIweightInitializerInhibitory, kernel_constraint=EIweightConstraintNegative, bias_constraint=EIbiasConstraintNegative)(h0I) #excitatory neuron inhibitory inputs
                        h1Ie = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=EIweightInitializerExcitatory, kernel_constraint=EIweightConstraintPositive, bias_constraint=EIbiasConstraintPositive)(h0E) #inhibitory neuron excitatory inputs
                        h1Ii = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=EIweightInitializerInhibitory, kernel_constraint=EIweightConstraintNegative, bias_constraint=EIbiasConstraintNegative)(h0I) #inhibitory neuron inhibitory inputs
                        h1E = tf.keras.layers.Add()([h1Ee, h1Ei])
                        h1I = tf.keras.layers.Add()([h1Ie, h1Ii])
                    h1E = tf.keras.layers.Activation(EIactivation)(h1E)
                    h1I = tf.keras.layers.Activation(EIactivation)(h1I)
                    h1 = (h1E, h1I)
        else:
            h1I = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=EIweightInitializerExcitatory, kernel_constraint=EIweightConstraintPositive, bias_constraint=EIbiasConstraintPositive)(h0) #inhibitory interneuron (excitatory inputs)
            h1I = tf.keras.layers.Activation(EIactivation)(h1I) #not required
            h1I = h1I*calculateInhibitoryNeuronNormalisationFactor(h0, h1I)
            h1Ee = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=EIweightInitializerExcitatory, kernel_constraint=EIweightConstraintPositive, bias_constraint=EIbiasConstraintPositive)(h0) #excitatory neuron excitatory inputs
            h1Ei = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=EIweightInitializerInhibitory, kernel_constraint=EIweightConstraintNegative, bias_constraint=EIbiasConstraintNegative)(h1I) #excitatory neuron inhibitory inputs
            h1E = tf.keras.layers.Add()([h1Ee, h1Ei])
            h1E = tf.keras.layers.Activation(EIactivation)(h1E)
            h1 = h1E
    return h1

def calculateInhibitoryNeuronNormalisationFactor(h0, h1I):
    h1InormalisationFactor = tf.reduce_mean(h0)/tf.reduce_mean(h1I)
    return h1InormalisationFactor

def concatEIneurons(h):
    if(inlineImplementation):
        if(positiveWeightImplementation):
            return h
        else: 
            if(integrateWeights):
                pass
            else:
                hE, hI = h
                h = tf.keras.layers.Concatenate()([hE, hI])
            return h
    else:
        return h

x = tf.keras.layers.Input(shape=input_shape)
h0 = tf.keras.layers.Flatten()(x)

hLast = h0
if(numberOfHiddenLayers >= 1):
    h1 = createEIlayer(1, h0, firstLayer=True)
    hLast = h1
if(numberOfHiddenLayers >= 2):
    h2 = createEIlayer(2, h1)
    hLast = h2
if(numberOfHiddenLayers >= 3):
    h3 = createEIlayer(3, h2)
    hLast = h3
if(numberOfHiddenLayers >= 4):
    h4 = createEIlayer(4, h3)
    hLast = h4
if(addSkipLayers):
    mList = []
    if(numberOfHiddenLayers >= 1):
        m1 = tf.keras.layers.Flatten()(concatEIneurons(h1))
        mList.append(m1)
    if(numberOfHiddenLayers >= 2):
        m2 = tf.keras.layers.Flatten()(concatEIneurons(h2))
        mList.append(m2)
    if(numberOfHiddenLayers >= 3):
        m3 = tf.keras.layers.Flatten()(concatEIneurons(h3))
        mList.append(m3)
    if(numberOfHiddenLayers >= 4):
        m4 = tf.keras.layers.Flatten()(concatEIneurons(h4))
        mList.append(m4)
    hLast = tf.keras.layers.concatenate(mList)
else:
    hLast = concatEIneurons(hLast)

if(generateUntrainedNetwork):
    hLast = tf.keras.layers.Lambda(lambda x: tf.keras.backend.stop_gradient(x))(hLast)
y = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_constraint=EIweightConstraintLastLayer, bias_constraint=EIbiasConstraintLastLayer)(hLast)
model = tf.keras.Model(x, y)

#print(model.summary())
#model.compile(optimizer=tf.keras.optimizers.RMSprop(epsilon=1e-08), loss='categorical_crossentropy', metrics=['acc'])
#evaluation accuracy: ? (with 1 or 2 hidden layers)


In [124]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [126]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

print(model.summary())
#printModelSummary(model)


Model: "model_329"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 28, 28)]     0           []                               
                                                                                                  
 flatten_7 (Flatten)            (None, 784)          0           ['input_8[0][0]']                
                                                                                                  
 dense_49 (Dense)               (None, 128)          100480      ['flatten_7[0][0]']              
                                                                                                  
 activation_28 (Activation)     (None, 128)          0           ['dense_49[0][0]']               
                                                                                          

In [127]:
if(debugPreTrainWeights):
    testwritefile = open('weightsPreTrain.txt', 'w')
    for layerIndex, layer in enumerate(model.layers):
        heading = "\n" + "layer = " + str(layerIndex) + "\n"
        testwritefile.write(heading)
        #print(heading)
        weights = layer.get_weights()
        #print(weights)
        weightsS =  str(weights)
        testwritefile.write(weightsS)
    testwritefile.close()

In [128]:
if(debugPreTrainOutputs):
    testwritefile = open('outputPreTrain.txt', 'w')
    xTrainFirstSample = np.expand_dims(x_train[0], axis=0)
    for layerIndex, layer in enumerate(model.layers):
        heading = "\n" + "layer = " + str(layerIndex) + "\n"
        #print(heading)
        testwritefile.write(heading)
        func = K.function([model.get_layer(index=0).input], layer.output)
        layerOutput = func([xTrainFirstSample])  # input_data is a numpy array
        #print("layerOutput.shape = ", layerOutput.shape)
        layerOutputS =  str(layerOutput)
        testwritefile.write(layerOutputS)
    testwritefile.close()

## Train model

In [129]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/100
938/938 [==============================] - 4s 3ms/step - loss: 18.9159 - accuracy: 0.6583
Epoch 2/100
938/938 [==============================] - 3s 3ms/step - loss: 3.7561 - accuracy: 0.8192
Epoch 3/100
938/938 [==============================] - 3s 3ms/step - loss: 3.5708 - accuracy: 0.8378
Epoch 4/100
938/938 [==============================] - 3s 3ms/step - loss: 3.2515 - accuracy: 0.8554
Epoch 5/100
938/938 [==============================] - 3s 3ms/step - loss: 2.9038 - accuracy: 0.8675
Epoch 6/100
938/938 [==============================] - 3s 3ms/step - loss: 2.6913 - accuracy: 0.8754
Epoch 7/100
938/938 [==============================] - 3s 3ms/step - loss: 2.4172 - accuracy: 0.8782
Epoch 8/100
938/938 [==============================] - 3s 3ms/step - loss: 2.2339 - accuracy: 0.8844
Epoch 9/100
938/938 [==============================] - 3s 3ms/step - loss: 1.8655 - accuracy: 0.8935
Epoch 10/100
938/938 [==============================] - 3s 3ms/step - loss: 1.6093 - accur

In [130]:
if(debugPostTrainWeights):
    testwritefile = open('weightsPostTrain.txt', 'w')
    for layerIndex, layer in enumerate(model.layers):
        heading = "\n" + "layer = " + str(layerIndex) + "\n"
        testwritefile.write(heading)
        #print(heading)
        weights = layer.get_weights()
        #print(weights)
        weightsS =  str(weights)
        testwritefile.write(weightsS)
    testwritefile.close()

In [131]:
if(debugPostTrainOutputs):
    testwritefile = open('outputPostTrain.txt', 'w')
    xTrainFirstSample = np.expand_dims(x_train[0], axis=0)
    for layerIndex, layer in enumerate(model.layers):
        heading = "\n" + "layer = " + str(layerIndex) + "\n"
        #print(heading)
        testwritefile.write(heading)
        func = K.function([model.get_layer(index=0).input], layer.output)
        layerOutput = func([xTrainFirstSample])  # input_data is a numpy array
        #print(layerOutput)
        layerOutputS = str(layerOutput)  #tf.tensor.toString(layerOutput)    #layerOutput.tostring()
        testwritefile.write(layerOutputS)
    testwritefile.close()

## Evaluate model

In [132]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.1893 - accuracy: 0.9589 - 674ms/epoch - 2ms/step


[0.1892823576927185, 0.958899974822998]

In [133]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [134]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.08533758, 0.08533758, 0.08534266, 0.08533809, 0.08533758,
        0.08533758, 0.08533758, 0.23195612, 0.08533765, 0.08533762],
       [0.08533712, 0.08533733, 0.23196337, 0.08533712, 0.08533712,
        0.08533712, 0.08533712, 0.08533712, 0.08533946, 0.08533712],
       [0.08533711, 0.23196352, 0.08533711, 0.08533711, 0.08533711,
        0.08533711, 0.08533711, 0.08533889, 0.08533781, 0.08533711],
       [0.23196234, 0.08533719, 0.08533719, 0.08533719, 0.08533724,
        0.08533719, 0.08534002, 0.08533719, 0.08533719, 0.0853373 ],
       [0.08534024, 0.08534024, 0.08534024, 0.08534024, 0.23191398,
        0.08534024, 0.08534024, 0.08534025, 0.08534024, 0.08536408]],
      dtype=float32)>